## Bagging

The bagging estimate is defined by 
$$\hat{f}(x)=\frac{1}{B}\sum^{B}_{b=1}\hat{f}^{*b}(x)$$

Bagging seems to work especially well for high-variance, low-bias procedures, such as trees. The essential idea in bagging is to average noisy but aprroximately unbiased models, and hence reduce the variance.


## Bootstrap Methods
Suppose we have a model fit to set of training data. We denote the training set by $Z = (z_1, z_2, ..., z_N)$ where $z_i = (x_i, y_i)$. The basic idea is to randomly draw datasets with replacement from the training data, each sample the same size as the orignal training set. This is done $B$ times, producing B bootstrap datasets. Then we refit the model to each of the bootstrap datasets, and examine the behavior of the fits over the B replications.

## Random Forest for Regression or Classfication

1. For b = 1 to $B$:  
    (a) Draw a bootstrap sample **Z**^\* size *N* from the training data.  
    (b) Grow a random-forest tree $T_b$ to the bootstrapped data, by recursively repeating the following steps for each terminal node of the tree, until the minimum node size $n_{min}$ is reached.  
        i. Select m variables at random from the p variables.  
        ii. Pick the best variable/split-point among the m.  
        iii. Split the node into two daughter nodes.    
2. Output the ensemble of trees ${T_b}^B_1$.  

To make a prediction at a new point x:  
*Regression*: $\hat{f}^B_{rf}(x) = \frac{1}{B}\sum^{B}_{b=1}T_b(x)$.  
*Classification*: Let $\hat{C}_b(x)$ be the class prediction of the *b*th random forest tree. Then $\hat{C}_{rf}(x) = majority vote{}$